In [1]:
import ctypes
import ctypes.util
import numpy
import time
import math
import h5py
import warnings
import os
import os.path
import errno
import sys
import scipy.ndimage
import warnings
import gc
import nidaqmx
import os
import os.path
from os import path

In [ ]:
#Currently, lsfmx defines yTiles such that the maximum y coordinate imaged is typically LESS than the maximum y coordinate specified (indexing error)
#Goal is to fix this problem. imaging should cover all coordinates specified by user, and image more coordinates to achieve this if necessary

## Current code from lsm20X_LB_run.py

In [8]:
int(math.ceil(yLength/yWidth))

3

In [11]:
yLength/yWidth

2.8372781065088746

In [19]:
(24.56-20.61)/yWidth

2.3372781065088755

In [17]:
yMin+2*yWidth

-21.18

In [26]:
#yMax = 1, but maximum y coordinate imaged is actually 0.777
#this is because 1 is technically included in the frame when y = 0.77
#but this is not really what the user wants when specifying limits
xMin = 7.45#
xMax = 10.35 #4.98 #7.78 # #needs to be at least 2mm i think?
# ycenter = -14.11
yMin = 2.19 #ycenter - .385
yMax = 4.56 #-18.72 #-7.31 #-2.73 #ycenter +.7
zMin = -2.78 # #zcenter - .01
zMax = -2.38 #-3.24 # #zcenter + .01 #if these numbers do not make sense, might get error "idx was referenced before assignment" #if these numbers do not make sense, might get error "idx was referenced before assignment"

#for ECi
#Dimensions of FOV
xWidth = 0.209 #in um, not mm (2nd gen system 0.48) 300um/1427px   --> 400um/1912px
yWidth = 0.385 # mm camera's horizontal FOV
zWidth = 0.03 #3 # mm calculated based on 610px/125um 11-20-19(2nd gen system 0.07)this will directly change how far z-strips are placed from each otehr in big stitcher
# LB added lines below 1/4/20: User-defined spacing between image tiles to optimize stitching. by default they should be xWidth, yWidth, and zWidth
yStitchOverlay = 0.3825 #User-defined spacing between image tiles horizontally, will affect XML file but not imaging
zStitchOverlay = .03 #User-defined spacing between image tiles vertically, will affect XML file but not imaging
 
camY = 256 # pixels #"Z" direction, vertical in camera's FOV
camX = 2048 # pixels "Y" direction, horizontal in camera's FOV
expTime = 10.0 #4.99 ms
camOffset = 0.0 # counts
wavelengths = numpy.array([488,660]) # lambda in nm
laser_powers = numpy.array([1,2]) #filter 1: 685+ filter 2: LP + single-band filter collects 495-552nm
galvoXoffset = 2.82 #in V
galvoYoffset = numpy.array([-.92,-.925]) #for 660nm, 488nm respectively. usually these numbers should be the same for 20X imaging

galvoXamp = 1.6 # in V. should be 1.5V for 20X imaging
galvofreq = 1000 #in Hz. rule of thumb should tune to whatever minimum frequency looks "smooth" on the live camera feed
binning = '1x1'
flatField = 0

######### PARAMETERS ###########
xMin = xMin - .05 #extra room for safety
xMax = xMax + .05
#y min = ymin-ywidth/2 and ymax = ymax + ywidth makes it so that it'll tile 1 tile to the left and right of your center
yMin = yMin #- yWidth/2 #to account for FOV/2 width not accounted for when estimating tissue bounds, + a little extra
yMax = yMax + yWidth #/2

xLength = xMax - xMin # mm
yLength = yMax - yMin #LB commented math.ceil((yMax - yMin)/yWidth)*yWidth # mm
zLength = math.ceil((zMax - zMin)/zWidth)*zWidth # mm
volume = xLength*yLength*zLength
#Goal with yTiles is to cover whole area that user entered, + image extra if necessary
yTiles = int(math.ceil(yLength/yWidth)) #consider adding 1 to this to fix error #previously int(round(yLength/yWidth))
zTiles = int(round(zLength/zWidth))
nTiles = zTiles*yTiles*len(wavelengths)
# print('tissue volume = ' + str(volume) + 'mm^3')
# print('number of tiles = ' + str(nTiles))
xOff = xMax - (xLength)/2
yOff = yMax - yLength/2
zOff = zMin
nFrames = int(round(xLength/(xWidth/1000))) #number of frames in X direction
###want to make sure that structure at center of coordinates will be imaged at center of FOV

for j in range(zTiles): 
    for k in range(yTiles):
        for i in range(len(wavelengths)):
            idx = k+j*yTiles+i*yTiles*zTiles
            # print(idx)
            yPos = yOff- yLength/2.0 + k*yWidth #+ yWidth/2.0
#             print('y = ' + str(yPos))
            zPos = j*zWidth + zOff
            # print('z = ' + str(zPos))
print('User-defined ymax = ' + str(yMax - yWidth))       
print(yPos)

User-defined ymax = 4.56
4.5


## testing

In [13]:
#yMax = 1, but maximum y coordinate imaged is actually 0.777
#this is because 1 is technically included in the frame when y = 0.77
#but this is not really what the user wants when specifying limits
xMin = -9.60#
xMax = -7.28 #4.98 #7.78 # #needs to be at least 2mm i think?
# ycenter = -14.11
yMin = -22.55 #ycenter - .385
yMax = -18.50 #
zMin = -2.37 # #zcenter - .01
zMax = -2.34 #-3.24 # #zcenter + .01 #if these numbers do not make sense, might get error "idx was referenced before assignment" #if these numbers do not make sense, might get error "idx was referenced before assignment"

#for ECi
#Dimensions of FOV
xWidth = 0.209 #in um, not mm (2nd gen system 0.48) 300um/1427px   --> 400um/1912px
yWidth = 0.385 # mm camera's horizontal FOV
zWidth = 0.03 #3 # mm calculated based on 610px/125um 11-20-19(2nd gen system 0.07)this will directly change how far z-strips are placed from each otehr in big stitcher
# LB added lines below 1/4/20: User-defined spacing between image tiles to optimize stitching. by default they should be xWidth, yWidth, and zWidth
yStitchOverlay = 0.3825 #User-defined spacing between image tiles horizontally, will affect XML file but not imaging
zStitchOverlay = .03 #User-defined spacing between image tiles vertically, will affect XML file but not imaging
 
camY = 256 # pixels #"Z" direction, vertical in camera's FOV
camX = 2048 # pixels "Y" direction, horizontal in camera's FOV
expTime = 10.0 #4.99 ms
camOffset = 0.0 # counts
wavelengths = numpy.array([488,660]) # lambda in nm
laser_powers = numpy.array([1,2]) #filter 1: 685+ filter 2: LP + single-band filter collects 495-552nm
galvoXoffset = 2.82 #in V
galvoYoffset = numpy.array([-.92,-.925]) #for 660nm, 488nm respectively. usually these numbers should be the same for 20X imaging

galvoXamp = 1.6 # in V. should be 1.5V for 20X imaging
galvofreq = 1000 #in Hz. rule of thumb should tune to whatever minimum frequency looks "smooth" on the live camera feed
binning = '1x1'
flatField = 0

######### PARAMETERS ###########
xMin = xMin - .05 #extra room for safety
xMax = xMax + .05
#y min = ymin-ywidth/2 and ymax = ymax + ywidth makes it so that it'll tile 1 tile to the left and right of your center
yMin = yMin #- yWidth/2 #to account for FOV/2 width not accounted for when estimating tissue bounds, + a little extra
yMax = yMax + yWidth/2

xLength = xMax - xMin # mm
yLength = yMax - yMin #LB commented math.ceil((yMax - yMin)/yWidth)*yWidth # mm
zLength = math.ceil((zMax - zMin)/zWidth)*zWidth # mm
volume = xLength*yLength*zLength
#Goal with yTiles is to cover whole area that user entered, + image extra if necessary
yTiles = int(math.ceil(yLength/yWidth)) #consider adding 1 to this to fix error #previously int(round(yLength/yWidth))
zTiles = int(round(zLength/zWidth))
nTiles = zTiles*yTiles*len(wavelengths)
print('tissue volume = ' + str(volume) + 'mm^3')
print('number of tiles = ' + str(nTiles))
xOff = xMax - (xLength)/2
yOff = yMax - yLength/2
zOff = zMin
nFrames = int(round(xLength/(xWidth/1000))) #number of frames in X direction
###want to make sure that structure at center of coordinates will be imaged at center of FOV

for j in range(zTiles): 
    for k in range(yTiles):
        for i in range(len(wavelengths)):
            idx = k+j*yTiles+i*yTiles*zTiles
            # print(idx)
            yPos = yOff- yLength/2.0 + k*yWidth #+ yWidth/2.0
            print('y = ' + str(yPos))
            zPos = j*zWidth + zOff
            # print('z = ' + str(zPos))

tissue volume = 0.616011mm^3
number of tiles = 48
y = -22.55
y = -22.55
y = -22.165
y = -22.165
y = -21.78
y = -21.78
y = -21.395
y = -21.395
y = -21.01
y = -21.01
y = -20.625
y = -20.625
y = -20.240000000000002
y = -20.240000000000002
y = -19.855
y = -19.855
y = -19.47
y = -19.47
y = -19.085
y = -19.085
y = -18.7
y = -18.7
y = -18.315
y = -18.315
y = -22.55
y = -22.55
y = -22.165
y = -22.165
y = -21.78
y = -21.78
y = -21.395
y = -21.395
y = -21.01
y = -21.01
y = -20.625
y = -20.625
y = -20.240000000000002
y = -20.240000000000002
y = -19.855
y = -19.855
y = -19.47
y = -19.47
y = -19.085
y = -19.085
y = -18.7
y = -18.7
y = -18.315
y = -18.315


In [11]:
yLength/yWidth

2.5974025974025974